In [3]:
import h5py
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

# Find the repo root (parent of current folder)
repo_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')) if '__file__' not in globals() else os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(repo_root)

# Add only immediate subfolders of the repo root to sys.path (much faster)
for item in os.listdir(repo_root):
    item_path = os.path.join(repo_root, item)
    if os.path.isdir(item_path):
        sys.path.append(item_path)

from read_scope_data import read_hdf5_scope_data, read_hdf5_scope_tarr, read_hdf5_all_scopes_channels
from read_hdf5 import print_hdf5_info, print_scope_channel_descriptions
from process_xray_bdot_hdf5 import get_magnetron_power_data

%matplotlib qt

In [2]:
ifn = r"F:\AUG2025\P24\00_He1kG430G_5800A_K-25_2025-08-12.hdf5"
print_hdf5_info(ifn)

  creation_time: Tue Aug 12 15:24:40 2025
  description: Acquire scope and fast cam result with Tungsten dropper

LAPD B field:
Black magnets at south: PS12-13 -- 555 A
Yellow magnets: PS 1-4 -- 2600 A
Magenta magnets: PS 5,6,8,9,10 -- 910 A
Magenta magnets: PS 7 -- 265 A
Black magnet at north PS11: 3550 A

B profile (source: bulk : mirror min): 1000:1000:430 G

South LaB6 source:
He plasma, 130 V bank discharge voltage,  discharge current in filename comes from Labview display which is ~1kA lower than scope display
1/3 Hz rep rate
Heater: 2000 A
Gas puff using Piezo near south anode (~1" size cu tube feeding gas ~ x = 40 cm)
from east & west: 5ms, 66V, He 40PSI
from top: 20ms, 66V pulse
No MFC
Max Pressure:9e-6  torr (New Cold magnetron gauge P33 west, N2 calibration appears off)
Interferrometer @port 20, assuming 40 cm size plasma: 0.5-1e12 cm^-3 during experiment time
Interferrometer @port 29, assuming 40 cm size plasma: 0.5-1e12 cm^-3 during experiment time
No real time traces (int

In [ ]:
with h5py.File(ifn, 'r') as f:
    result = read_hdf5_all_scopes_channels(f, 1, include_tarr=True)

Scope 'Configuration' is not recorded for shot '1'
Scope 'Control' is not recorded for shot '1'
dict_keys(['time_array', 'channels'])
dict_keys(['time_array', 'channels'])
dict_keys(['time_array', 'channels'])


In [6]:
for keys in result.keys():
    scope = result[keys]
    print(scope['channels'].keys())

dict_keys(['C1', 'C2', 'C3'])
dict_keys(['C1', 'C2', 'C3', 'C4'])
dict_keys(['C2'])


In [ ]:
P_data, tarr, I_data, V_data, Pref_data = get_magnetron_power_data(ifn, 1)
plt.figure()
plt.plot(tarr*1e6, P_data)

Found current channel: C2 - magnetron current 0.25A/V
Found reflected power channel: C3 - reflected power (diode + 26dB)
Found voltage channel: C4 - magnetron voltage (X1000 probe)
Applying current scaling factor: 0.25 A/V
Magnetron power calculated for shot 1


In [4]:
print_scope_channel_descriptions(ifn, scope_name='bdotscope')


=== Channel Descriptions for bdotscope ===
  C1: Bx (X100) @P14
  C2: Bx (X100) @P20
  C3: Bx (X100) @P28


In [ ]:
scope_name = 'xrayscope'
channel_name = 'C2'
shot_number = 1
data = read_hdf5_scope_data(ifn, scope_name, channel_name, shot_number)
tarr = read_hdf5_scope_tarr(ifn, scope_name)

In [ ]:
plt.figure()
plt.plot(tarr*1e3, data)

In [ ]:
# Test the updated process_shot_bdot function
from process_xray_bdot_hdf5 import process_shot_bdot

# Test the function with the HDF5 file
stft_time, freq, stft_matrix1, stft_matrix2, stft_matrix3, channel_info = process_shot_bdot(ifn, shot_number=1, debug=True)

# Print channel information
print("\n=== Channel Information ===")
for channel, info in channel_info.items():
    field_comp = info['field_component']
    position = info['position']
    desc = info['description']
    print(f"{channel}: {field_comp} at {position} - {desc}")

# Check if data was successfully read
print(f"\nSTFT matrices availability:")
print(f"Matrix 1 (C1): {'Available' if stft_matrix1 is not None else 'None'}")
print(f"Matrix 2 (C2): {'Available' if stft_matrix2 is not None else 'None'}")
print(f"Matrix 3 (C3): {'Available' if stft_matrix3 is not None else 'None'}")

if stft_time is not None:
    print(f"STFT time array shape: {stft_time.shape}")
    print(f"Frequency array shape: {freq.shape}")